# HELP - Bootes Master List Creation

This notebook presents the creation of the HELP master list on the Bootes field. This field was originally ingested into HeDAM based on a masterlist produced by Eduardo Gonzales-Solares. Ken Duncan subsequently retrieved Michael Brown's precompiled masterlist for the Bootes field and therefore this notebook only contains some simple naming changes and astrometry correction. The original catalogue is described in DMU_0.

In [ ]:
from herschelhelp_internal import git_version
print("This notebook was run with herschelhelp_internal version: \n{}".format(git_version()))

In [ ]:
%matplotlib inline
#%config InlineBackend.figure_format = 'svg'

import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))

from collections import OrderedDict
import os

from astropy import units as u
from astropy.coordinates import SkyCoord
from astropy.table import Column, Table
import numpy as np

from herschelhelp_internal.flagging import  gaia_flag_column
from herschelhelp_internal.masterlist import nb_astcor_diag_plot, remove_duplicates
from herschelhelp_internal.utils import astrometric_correction, mag_to_flux

In [ ]:
OUT_DIR =  os.environ.get('TMP_DIR', "./data_tmp")
try:
    os.makedirs(OUT_DIR)
except FileExistsError:
    pass

RA_COL = "bootes_ra"
DEC_COL = "botes_dec"

## I - Column selection

In [ ]:
imported_columns = OrderedDict({
        "objID": "ps1_id",
        "raMean": "ps1_ra",
        "decMean": "ps1_dec",
        "gApMag": "m_ap_gpc1_g",
        "gApMagErr": "merr_ap_gpc1_g",
        "gKronMag": "m_gpc1_g",
        "gKronMagErr": "merr_gpc1_g",
        "rApMag": "m_ap_gpc1_r",
        "rApMagErr": "merr_ap_gpc1_r",
        "rKronMag": "m_gpc1_r",
        "rKronMagErr": "merr_gpc1_r",
        "iApMag": "m_ap_gpc1_i",
        "iApMagErr": "merr_ap_gpc1_i",
        "iKronMag": "m_gpc1_i",
        "iKronMagErr": "merr_gpc1_i",
        "zApMag": "m_ap_gpc1_z",
        "zApMagErr": "merr_ap_gpc1_z",
        "zKronMag": "m_gpc1_z",
        "zKronMagErr": "merr_gpc1_z",
        "yApMag": "m_ap_gpc1_y",
        "yApMagErr": "merr_ap_gpc1_y",
        "yKronMag": "m_gpc1_y",
        "yKronMagErr": "merr_gpc1_y"
    })


catalogue = Table.read("../../dmu0/dmu0_PanSTARRS1-3SS/data/PanSTARRS1-3SS_XMM-LSS.fits")[list(imported_columns)]
for column in imported_columns:
    catalogue[column].name = imported_columns[column]

epoch = 2012

# Clean table metadata
catalogue.meta = None

In [ ]:
# Adding flux and band-flag columns
for col in catalogue.colnames:
    if col.startswith('m_'):
        
        errcol = "merr{}".format(col[1:])
        
        # -999 is used for missing values
        catalogue[col][catalogue[col] < -900] = np.nan
        catalogue[errcol][catalogue[errcol] < -900] = np.nan     
        
        flux, error = mag_to_flux(np.array(catalogue[col]), np.array(catalogue[errcol]))
        
        # Fluxes are added in µJy
        catalogue.add_column(Column(flux * 1.e6, name="f{}".format(col[1:])))
        catalogue.add_column(Column(error * 1.e6, name="f{}".format(errcol[1:])))
        
        # Band-flag column
        if "ap" not in col:
            catalogue.add_column(Column(np.zeros(len(catalogue), dtype=bool), name="flag{}".format(col[1:])))
        
# TODO: Set to True the flag columns for fluxes that should not be used for SED fitting.

In [ ]:
catalogue[:10].show_in_notebook()

## II - Removal of duplicated sources

We remove duplicated objects from the input catalogues.

In [ ]:
SORT_COLS = ['merr_ap_gpc1_r', 'merr_ap_gpc1_g', 'merr_ap_gpc1_i', 'merr_ap_gpc1_z', 'merr_ap_gpc1_y']
FLAG_NAME = 'ps1_flag_cleaned'

nb_orig_sources = len(catalogue)

catalogue = remove_duplicates(catalogue, RA_COL, DEC_COL,  sort_col=SORT_COLS, flag_name=FLAG_NAME)

nb_sources = len(catalogue)

print("The initial catalogue had {} sources.".format(nb_orig_sources))
print("The cleaned catalogue has {} sources ({} removed).".format(nb_sources, nb_orig_sources - nb_sources))
print("The cleaned catalogue has {} sources flagged as having been cleaned".format(np.sum(catalogue[FLAG_NAME])))

## III - Astrometry correction

We match the astrometry to the Gaia one. We limit the Gaia catalogue to sources with a g band flux between the 30th and the 70th percentile. Some quick tests show that this give the lower dispersion in the results.

In [ ]:
gaia = Table.read("../../dmu0/dmu0_GAIA/data/GAIA_XMM-LSS.fits")
gaia_coords = SkyCoord(gaia['ra'], gaia['dec'])

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

In [ ]:
delta_ra, delta_dec =  astrometric_correction(
    SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]),
    gaia_coords
)

print("RA correction: {}".format(delta_ra))
print("Dec correction: {}".format(delta_dec))

In [ ]:
catalogue[RA_COL] +=  delta_ra.to(u.deg)
catalogue[DEC_COL] += delta_dec.to(u.deg)

In [ ]:
nb_astcor_diag_plot(catalogue[RA_COL], catalogue[DEC_COL], 
                    gaia_coords.ra, gaia_coords.dec)

## IV - Flagging Gaia objects

In [ ]:
catalogue.add_column(
    gaia_flag_column(SkyCoord(catalogue[RA_COL], catalogue[DEC_COL]), epoch, gaia)
)

In [ ]:
GAIA_FLAG_NAME = "ps1_flag_gaia"

catalogue['flag_gaia'].name = GAIA_FLAG_NAME
print("{} sources flagged.".format(np.sum(catalogue[GAIA_FLAG_NAME] > 0)))

# V - Saving to disk

In [ ]:
catalogue.write("{}/PS1.fits".format(OUT_DIR), overwrite=True)